In [ ]:
#meta 5/30/2021 Google I/O TFHub Example - Retraining an Image Classifier (Transfer Learning)
#origin: 
#Session: TensorFlow Hub for Real World Impact  5/19/2021
#example: iBean image classifier w/ Cassava leaf pretrained model

#ref
# TF Hub for TF2: Retraining an image classifier
#https://colab.research.google.com/github/tensorflow/hub/blob/master/examples/colab/tf2_image_retraining.ipynb
# Transfer learning with TensorFlow Hub
#https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/transfer_learning_with_hub.ipynb#scrollTo=mqnsczfLgcwv

#history
#5/30/2021 IMAGE CLASSIFIER w/TRANSFER LEARNING
#     Google I/O example
#     https://events.google.com/io/session/d39de9e5-e54f-48eb-acf7-5a5a35a2cb3e?utm_medium=referral&utm_campaign=io21-resources&lng=en 
#     iBean image classifier w/ Cassava leaf pretrained model

# TFHub Example - Retraining an Image Classifier
Image classification w/ transfer learning

Example: iBean image classifier w/ Cassava leaf pretrained model

In [1]:
import itertools
import os

import matplotlib.pylab as plt
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub

print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.test.is_gpu_available() else "NOT AVAILABLE")

TF version: 2.4.1
Hub version: 0.12.0
GPU is available


## 0. Get Data
Data repo for the ibean project of the AIR lab.

In [2]:
import tensorflow_datasets as tfds
datasets, info = tfds.load(name = 'beans', with_info=True, as_supervised=True)
num_classes = info.features['label'].num_classes

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

/opt/conda/lib/python3.7/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'storage.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/opt/conda/lib/python3.7/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'storage.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,
/opt/conda/lib/python3.7/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'storage.googleapis.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/beans/0.1.0.incompleteBRIJIA/beans-train.tfrecord


...

  0%|          | 0/133 [00:00<?, ? examples/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/beans/0.1.0.incompleteBRIJIA/beans-test.tfrecord


  0%|          | 0/128 [00:00<?, ? examples/s]

Dataset beans downloaded and prepared to /root/tensorflow_datasets/beans/0.1.0. Subsequent calls will reuse this data.


In [3]:
info

tfds.core.DatasetInfo(
    name='beans',
    version=0.1.0,
    description='Beans is a dataset of images of beans taken in the field using smartphone
cameras. It consists of 3 classes: 2 disease classes and the healthy class.
Diseases depicted include Angular Leaf Spot and Bean Rust. Data was annotated
by experts from the National Crops Resources Research Institute (NaCRRI) in
Uganda and collected by the Makerere AI research lab.',
    homepage='https://github.com/AI-Lab-Makerere/ibean/',
    features=FeaturesDict({
        'image': Image(shape=(500, 500, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=3),
    }),
    total_num_examples=1295,
    splits={
        'test': 128,
        'train': 1034,
        'validation': 133,
    },
    supervised_keys=('image', 'label'),
    citation="""@ONLINE {beansdata,
        author="Makerere AI Lab",
        title="Bean disease dataset",
        month="January",
        year="2020",
        url="https://git

## 1. Prep Data

In [4]:
def preprocess_fn(image, label):
    #normalize [0,255] to [0,1]
    image = tf.cast(image, tf.float32) / 255.
    #resize the image to 224x224
    image = tf.image.resize(image, (224, 224))
    return image, label

train_dataset = datasets['train'].map(preprocess_fn).batch(10)
valid_dataset = datasets['validation'].map(preprocess_fn).batch(10)

len(train_dataset), len(valid_dataset)

(104, 14)

## 2. Train Model w/ Transfer Learning

In [5]:
model = tf.keras.Sequential([
    hub.KerasLayer('https://tfhub.dev/google/cropnet/feature_vector/cassava_disease_V1/1'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes)
    
])

model.compile(
  optimizer=tf.keras.optimizers.Adam(), 
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])


model.fit(train_dataset, epochs = 10, validation_data = valid_dataset)

Epoch 1/10
104/104 [==============================] - 17s 61ms/step - loss: 0.6571 - accuracy: 0.7074 - val_loss: 0.1519 - val_accuracy: 0.9624
Epoch 2/10
104/104 [==============================] - 3s 29ms/step - loss: 0.2213 - accuracy: 0.8983 - val_loss: 0.1143 - val_accuracy: 0.9774
Epoch 3/10
104/104 [==============================] - 3s 29ms/step - loss: 0.1455 - accuracy: 0.9344 - val_loss: 0.1498 - val_accuracy: 0.9474
Epoch 4/10
104/104 [==============================] - 3s 31ms/step - loss: 0.1223 - accuracy: 0.9545 - val_loss: 0.1822 - val_accuracy: 0.9323
Epoch 5/10
104/104 [==============================] - 3s 29ms/step - loss: 0.1134 - accuracy: 0.9613 - val_loss: 0.1689 - val_accuracy: 0.9474
Epoch 6/10
104/104 [==============================] - 3s 28ms/step - loss: 0.0919 - accuracy: 0.9714 - val_loss: 0.2326 - val_accuracy: 0.9023
Epoch 7/10
104/104 [==============================] - 3s 29ms/step - loss: 0.0934 - accuracy: 0.9641 - val_loss: 0.1060 - val_accuracy: 0.962

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              4234118   
_________________________________________________________________
dense (Dense)                (None, 64)                81984     
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 195       
Total params: 4,316,297
Trainable params: 82,179
Non-trainable params: 4,234,118
_________________________________________________________________


In [7]:
#save model
model_name = 'cassava_disease_V1'
saved_model_path = f"model_{model_name}"
tf.saved_model.save(model, saved_model_path) 

## 3. Evaluate Model
myAdd

In [9]:
test_dataset = datasets['test'].map(preprocess_fn).batch(10)
len(test_dataset)

13

In [13]:
#predict
y_hat = model.predict(test_dataset) 
print(y_hat.shape)
#view first 3 predictions
y_hat[:3]

(128, 3)


array([[ 11.30929  ,   6.211437 , -12.873492 ],
       [ -4.311848 ,  13.925394 ,  -2.8326743],
       [  2.209536 ,   7.8851194,  -6.267129 ]], dtype=float32)

In [21]:
predicted_index = np.argmax(y_hat, axis=1)
predicted_index

array([0, 1, 1, 1, 2, 0, 1, 0, 0, 1, 0, 1, 1, 2, 2, 2, 1, 2, 2, 1, 1, 1,
       2, 1, 2, 2, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 2, 0, 0, 2, 1, 0, 2,
       2, 1, 1, 0, 2, 0, 0, 1, 2, 2, 2, 2, 2, 1, 1, 0, 0, 0, 0, 1, 0, 1,
       1, 2, 0, 1, 0, 0, 0, 2, 1, 2, 1, 0, 2, 2, 2, 1, 1, 2, 0, 0, 2, 1,
       2, 1, 1, 1, 1, 1, 1, 1, 0, 2, 1, 1, 2, 2, 0, 1, 1, 2, 0, 1, 0, 0,
       1, 2, 2, 0, 0, 1, 2, 2, 1, 0, 0, 2, 0, 2, 1, 2, 0, 1])

In [24]:
#todo: compare to actual labels

In [ ]:
mystop

## $Xtra

In [ ]:
#$xtra only 48% accuracy
model = tf.keras.Sequential([
    hub.KerasLayer('https://tfhub.dev/google/cropnet/feature_vector/imagenet/1'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(num_classes)
    
])

model.compile(
  optimizer=tf.keras.optimizers.Adam(), 
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])


model.fit(train_dataset, epochs = 10, validation_data = valid_dataset)

model.summary()